In [241]:
import json
import requests
from pprint import pprint

In [242]:
endpoint = 'https://qnafaq-aslv4rot7gumsy4.search.windows.net/'
api_version = '?api-version=2019-05-06'
headers = {'Content-Type': 'application/json',
        'api-key': 'B08826F08F7F9A7C4B26F2D315FFFDD3' }
params = { 'api-version': '2019-05-06' }

In [243]:
url = endpoint + "indexes" + api_version + "&$select=name"
response  = requests.get(url, headers=headers)
index_list = response.json()
pprint(index_list)

{'@odata.context': 'https://qnafaq-aslv4rot7gumsy4.search.windows.net/$metadata#indexes(name)',
 'value': [{'name': 'margiestravelsearchindex'}]}


In [244]:
# Create a skillset
skillset_name = "ourcogskills"

skillset_payload = {
    "name": skillset_name,
    "description":
    "Extract entities, detect language and extract key-phrases",
    "skills":
    [
        
        {
          "description": "Extracts text (plain and structured) from image.",
          "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
          "context": "/document/normalized_images/*",
          "defaultLanguageCode": "en",
          "detectOrientation": True,
          "inputs": [
            {
              "name": "image",
              "source": "/document/normalized_images/*"
            }
          ],
          "outputs": [
            {
              "name": "text",
              "targetName": "ocr_text"
            },
            {
              "name": "layoutText",
              "targetName": "ocr_layout_text"
            }
          ]
        },
        {
          "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
          "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field.",
          "context": "/document",
          "insertPreTag": " ",
          "insertPostTag": " ",
          "inputs": [
            {
              "name":"text", "source": "/document/content"
            },
            {
              "name": "itemsToInsert", "source": "/document/normalized_images/*/ocr_text"
            }
           
          ],
          "outputs": [
            {
              "name": "mergedText", "targetName" : "merged_text"
            }
          ]
        },
        {
            "description": "Extract image analysis.",
            "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "visualFeatures": [                
                "Description"
            ],
            "inputs": [
                {
                    "name": "image",
                    "source": "/document/normalized_images/*"
                }
            ],
            "outputs": [
                {
                    "name": "description", "targetName": "image_description"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.EntityRecognitionSkill",
            "categories": ["Organization","Person","Location","Datetime", "URL"],
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text", "source": "/document/content"
                }
            ],
            "outputs": [
                {
                    "name": "organizations", "targetName": "organizations"
                },
                {
                    "name": "persons", "targetName": "people"
                },
                {
                    "name": "locations", "targetName": "locations"
                },
                {
                    "name": "dateTimes", "targetName": "dateTimes"
                } ,
                {
                    "name": "urls", "targetName": "urls"
                }

            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
            "inputs": [
              {
                "name": "text",
                "source": "/document/content"
              }
            ],
            "outputs": [
              {
                "name": "keyPhrases",
                "targetName": "myKeyPhrases"
              }
            ]
          },
        {
            "@odata.type": "#Microsoft.Skills.Text.SentimentSkill",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/content"
                }
                    ],
            "outputs": [
                {
                    "name": "score",
                    "targetName": "mySentiment"
                }
                    ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
            "description": "Our text analytics skill.",
            "uri": "https://hack14func.azurewebsites.net/api/WordStuff?code=hA2fi3TMWxAjGKyUpKyMMGcIxWGMSEPU7oyp0K4U6kbstZDZpaAK/A==",
              "inputs": [
                {
                  "name": "text",
                  "source": "/document/merged_text"
                }
              ],
              "outputs": [
                {
                  "name": "words",
                  "targetName": "frequentwords"
                }
              ]
          }
    ]
}

r = requests.put(endpoint + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)

204


In [245]:
index_name = "margiestravelsearchindex"

index_payload = {
    "name": index_name,
    "fields": [{
        "name": "id",
        "type": "Edm.String",
        "key": "true",
        "filterable": "false"
    }, {
        "name": "url",
        "type": "Edm.String",
        "searchable": "false",
        "filterable": "false",
        "sortable": "false",
        "facetable": "false"
    }, {
        "name": "file_name",
        "type": "Edm.String",
        "searchable": "true",
        "filterable": "false",
        "sortable": "false",
        "facetable": "false",
        "analyzer": "en.lucene"
    }, {
        "name": "content",
        "type": "Edm.String",
        "searchable": "true",
        "filterable": "false",
        "sortable": "false",
        "facetable": "false",
        "analyzer": "en.lucene",
        "synonymMaps": ["mysynonymmap"]
    }, {
        "name": "size",
        "type": "Edm.Int64",
        "searchable": "false",
        "filterable": "false",
        "sortable": "false",
        "facetable": "false"
    }, {
        "name": "last_modified",
        "type": "Edm.DateTimeOffset",
        "searchable": "false",
        "filterable": "true",
        "sortable": "false",
        "facetable": "false"
    }, {
        "name": "myKeyPhrases",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false",
        "synonymMaps": ["mysynonymmap"]
    }, {
        "name": "organizations",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false",
        "synonymMaps": ["mysynonymmap"]
    }, {
        "name": "locations",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false",
        "synonymMaps": ["mysynonymmap"]
    }, {
        "name": "people",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"
    }, {
        "name": "dateTimes",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "false",
        "facetable": "false"
    }, {
        "name": "urls",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "true",
        "facetable": "false"
    }, {
        "name": "mySentiment",
        "type": "Edm.Double",
        "searchable": "false",
        "sortable": "false",
        "filterable": "true",
        "facetable": "false"
    }, {
        "name": "image_description",
        "type": "Collection(Edm.ComplexType)",
        "fields": [{
            "name": "tags",
            "type": "Collection(Edm.String)",
            "searchable": True,
            "filterable": False,
            "facetable": False,
            "synonymMaps": ["mysynonymmap"]
        }, {
            "name": "captions",
            "type": "Collection(Edm.ComplexType)",
            "fields": [{
                "name": "text",
                "type": "Edm.String",
                "searchable": True,
                "filterable": False,
                "facetable": False,
                "synonymMaps": ["mysynonymmap"]
            }, {
                "name": "confidence",
                "type": "Edm.Double",
                "searchable": False,
                "filterable": False,
                "facetable": False
            }]
        }]
    }, {
        "name": "ocr_text",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "filterable": "false",
        "sortable": "false",
        "facetable": "false",
        "analyzer": "en.lucene",
        "synonymMaps": ["mysynonymmap"]
    }, {
        "name": "frequentwords",
        "type": "Collection(Edm.String)",
        "searchable": "true",
        "sortable": "false",
        "filterable": "true",
        "facetable": "false",
        "synonymMaps": ["mysynonymmap"]
    }],
    "scoringProfiles": [
        {
        "name": "jasonsScoringProfile",
        "functions": [
            {
                "type": "freshness",
                "fieldName": "last_modified",
                "boost": 10,
                "interpolation": "linear",
                "freshness": {
                    "boostingDuration": "P1D"
            }
        }
        ]
    }
    ], 
    "suggesters": [{
        "name": "suggester",
        "searchMode": "analyzingInfixMatching",
        "sourceFields": ["locations"]
    }]
}

In [246]:
r = requests.delete(endpoint + "/indexes/" + index_name, headers=headers, params=params)
print(r.status_code)

r = requests.put(endpoint + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.text)

204
201
{"@odata.context":"https://qnafaq-aslv4rot7gumsy4.search.windows.net/$metadata#indexes/$entity","@odata.etag":"\"0x8D73C9CE0B8D8D3\"","name":"margiestravelsearchindex","defaultScoringProfile":null,"fields":[{"name":"id","type":"Edm.String","searchable":true,"filterable":false,"retrievable":true,"sortable":true,"facetable":true,"key":true,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"synonymMaps":[]},{"name":"url","type":"Edm.String","searchable":false,"filterable":false,"retrievable":true,"sortable":false,"facetable":false,"key":false,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":null,"synonymMaps":[]},{"name":"file_name","type":"Edm.String","searchable":true,"filterable":false,"retrievable":true,"sortable":false,"facetable":false,"key":false,"indexAnalyzer":null,"searchAnalyzer":null,"analyzer":"en.lucene","synonymMaps":[]},{"name":"content","type":"Edm.String","searchable":true,"filterable":false,"retrievable":true,"sortable":false,"facetable":false,"key

In [247]:
synonym_payload = {
   "name":"mysynonymmap",
   "format":"solr",
   "synonyms": "USA, United States, America United States of America\nUK, United Kingdom, Britain, Great Britain\nUAE, United Arab Emirates, Emirates\n"
}

In [248]:
r = requests.post(endpoint + "/synonymmaps",
                 data=json.dumps(synonym_payload), headers=headers, params=params)
print(r.status_code)

400


In [249]:
synonym_payload


{'format': 'solr',
 'name': 'mysynonymmap',
 'synonyms': 'USA, United States, America United States of America\nUK, United Kingdom, Britain, Great Britain\nUAE, United Arab Emirates, Emirates\n'}

In [250]:
indexer_payload = {
    'name': index_name,
    'dataSourceName': 'blob-datasource',
    'skillsetName': skillset_name,
    'targetIndexName': index_name,
    'fieldMappings': [{
        "sourceFieldName": "metadata_storage_size",
        "targetFieldName": "size"
    }, {
        "sourceFieldName": "metadata_storage_path",
        "targetFieldName": "url"
    }, {
        "sourceFieldName": "metadata_storage_last_modified",
        "targetFieldName": "last_modified"
    }, {
        "sourceFieldName": "metadata_storage_name",
        "targetFieldName": "file_name"
    }, {
        "sourceFieldName": "metadata_storage_path",
        "targetFieldName": "id",
        "mappingFunction": {
            "name": "base64Encode"
        }
    }],
    "outputFieldMappings": [{
        "sourceFieldName": "/document/organizations",
        "targetFieldName": "organizations"
    }, {
        "sourceFieldName": "/document/people",
        "targetFieldName": "people"
    }, {
        "sourceFieldName": "/document/locations",
        "targetFieldName": "locations"
    }, {
        "sourceFieldName": "/document/dateTimes",
        "targetFieldName": "dateTimes"
    }, {
        "sourceFieldName": "/document/urls",
        "targetFieldName": "urls"
    }, {
        "sourceFieldName": "/document/myKeyPhrases",
        "targetFieldName": "myKeyPhrases"
    }, {
        "sourceFieldName": "/document/mySentiment",
        "targetFieldName": "mySentiment"
    },
        {
            "sourceFieldName": "/document/normalized_images/*/image_description",
            "targetFieldName": "image_description"
        }, 
        {
        "sourceFieldName": "/document/normalized_images/*/ocr_text",
        "targetFieldName": "ocr_text"
    },
    {
        "sourceFieldName": "/document/merged_text",
        "targetFieldName": "content"
    },
    {
        "sourceFieldName": "/document/frequentwords",
        "targetFieldName": "frequentwords"
    }],
    "parameters":
      {
        "configuration": 
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
      }
    
}

In [251]:
r = requests.delete(endpoint + "/indexers/" + index_name, headers=headers, params=params)
print(r.status_code)

r = requests.put(endpoint + "/indexers/" + index_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)

204
201


In [252]:
searchendpoint = "https://qnafaq-aslv4rot7gumsy4.search.windows.net/indexes/margiestravelsearchindex/docs/search?api-version=2019-05-06"
searchrequest =   {  "select":"file_name,frequentwords,content",
     "count": True,
      "filter": "search.ismatch('good','frequentwords')",
      "queryType": "full", 
      "searchMode": "all"                    
        }

r=requests.post(url=searchendpoint,json = searchrequest,headers=headers)
print(r)
r.text

<Response [200]>


'{"@odata.context":"https://qnafaq-aslv4rot7gumsy4.search.windows.net/indexes(\'margiestravelsearchindex\')/$metadata#docs(*)","@odata.count":0,"value":[]}'